In this notebook, we do the following:

1. Download the DEM that match the (union) of the bounds of the SLC data determined previously.
2. Downsample the DEM for geocoding

There are some minor details that are important as well. Since ISCE2 `dem.py` requires integer degrees for the bounding box, we have to overestimate the extent and then clip the raster.

ISCE2 is very particular about the DEM type it requires. So we have to create a VRT and then run `fixImageXML`. Not sure as to why, but it works after this.

In [2]:
from pathlib import Path
import rasterio
import numpy as np
import json
import site
import sys
import os
from subprocess import check_call
from rasterio.enums import Resampling

In [3]:
## Will be taken care of in papermill script
## If not, we must run jupyter from the `notebooks/` directory
if not os.environ.get('PGE_DIRECTORY'):
    path = Path(os.getcwd()).parents[0]
    os.environ['PGE_DIRECTORY'] = str(path.absolute())
    assert(path.name == 'alos2app_pge')

sys.path.append(os.environ['PGE_DIRECTORY'])
from ifg_tools.rio_tools import reproject_arr_to_match_profile

# Get Settings

In [2]:
from ifg_tools.UrlUtils import UrlUtils

uu = UrlUtils()

# Read Metdata

In [3]:
product_dir = list(Path('.').glob('ALOS2-GUNW-*'))[0]
prod_id = product_dir.name
prod_id

'ALOS2-GUNW-D-153-20150412-20150301-182629-18N-100W-v1_0_0'

In [4]:
metadata_path = product_dir/f'{prod_id}.met.json'
metadata = json.load(open(metadata_path, 'r'))
metadata

{'reference_metadata': {'slc_id': 'ALOS2047853215-150412',
  'production_level': 'WBDR1',
  'bbox_xy_coords': [[-99.54163571839928, 21.841270600310185],
   [-96.17730760260231, 21.25939696689257],
   [-96.843173553836, 18.068982542026124],
   [-100.13694111929185, 18.65929018004776]],
  'start_time': '2015-04-12T18:26:03.683087',
  'stop_time': '2015-04-12T18:26:55.705525',
  'absolute_orbit': '04785',
  'frame': '3215',
  'flight_direction': 'D',
  'satellite_name': 'ALOS2',
  'source': 'isce_preprocessing',
  'track': 153},
 'secondary_metadata': {'slc_id': 'ALOS2041643215-150301',
  'production_level': 'WBDR1',
  'bbox_xy_coords': [[-99.54225234548686, 21.84242172762871],
   [-96.17777138358394, 21.260454461136113],
   [-96.84383040237188, 18.07008971011115],
   [-100.13767953028737, 18.660475516270562]],
  'start_time': '2015-03-01T18:26:06.130416',
  'stop_time': '2015-03-01T18:26:58.152714',
  'absolute_orbit': '04164',
  'frame': '3215',
  'flight_direction': 'D',
  'satellite_n

# Download SRTMv3 DEM

In [5]:
isce_apps_path = site.getsitepackages()[0] + '/isce/applications'

In [6]:
bounds = metadata['bounds']
bounds

[-100.13767953028737,
 18.068982542026124,
 -96.17730760260231,
 21.84242172762871]

In [7]:
bounds_snwe = [bounds[k] for k in [1, 3, 0, 2]]
bounds_snwe = [np.floor(bounds_snwe[0]), np.ceil(bounds_snwe[1]),
               np.floor(bounds_snwe[2]), np.ceil(bounds_snwe[3]),
              ]
bounds_snwe = list(map(lambda x: f'{x:1.0f}', bounds_snwe))
bounds_snwe = ' '.join(bounds_snwe)
bounds_snwe

'18 22 -101 -96'

In [8]:
dem_url = uu['ARIA_DEM_URL']
dem_user = uu['ARIA_DEM_U']
dem_pass = uu['ARIA_DEM_P']

dem_cmd = [f"{isce_apps_path}/dem.py", 
           "-a",
           "stitch", 
           "-b", 
           bounds_snwe,
           "-k", 
           "-s", "1", 
           "-f", 
           "-x", 
           "-c", 
           "-n", dem_user, 
           "-w", dem_pass,
           "-u", dem_url
        ]
dem_cmd_line = " ".join(dem_cmd)

In [9]:
%%time

check_call(dem_cmd_line, shell=True)

CPU times: user 7.69 ms, sys: 11.2 ms, total: 18.9 ms
Wall time: 52.1 s


0

## Inspect the DEM

In [10]:
preprocess_dem_file = list(Path('.').glob("*.dem.wgs84"))[0]
preprocess_dem_file

PosixPath('demLat_N18_N22_Lon_W101_W096.dem.wgs84')

In [11]:
with rasterio.open(f'{preprocess_dem_file}.vrt') as ds:
    dem_profile = ds.profile
    
dem_profile

{'driver': 'VRT', 'dtype': 'int16', 'nodata': None, 'width': 18000, 'height': 14400, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0002777777777777778, 0.0, -101.0,
       0.0, -0.0002777777777777778, 22.0), 'tiled': False}

# Fix Image XML

In [12]:
fix_cmd = [f"{isce_apps_path}/fixImageXml.py",
           "-i", str(preprocess_dem_file), 
           "--full"
           ]
fix_cmd_line = " ".join(fix_cmd)
fix_cmd_line

'/u/leffe-data2/cmarshak/miniconda3/envs/isce2/lib/python3.8/site-packages/isce/applications/fixImageXml.py -i demLat_N18_N22_Lon_W101_W096.dem.wgs84 --full'

In [13]:
check_call(fix_cmd_line, shell=True)

0

# Translate

In [16]:
DRIVER = 'ISCE'

In [17]:
with rasterio.open(preprocess_dem_file) as ds:
    dem_profile = ds.profile
    dem = ds.read(1)
    
dem_profile

{'driver': 'ISCE', 'dtype': 'int16', 'nodata': None, 'width': 18000, 'height': 14400, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0002777777777777778, 0.0, -101.0,
       0.0, -0.0002777777777777778, 22.0), 'tiled': False}

In [18]:
dem_profile_trans = dem_profile.copy()
dem_profile_trans['driver'] = DRIVER

In [19]:
with rasterio.open('preprocess_dem.wgs84', 'w', **dem_profile_trans) as ds:
    ds.write(dem, 1)

## Downsample DEM

Source: https://rasterio.readthedocs.io/en/latest/topics/resampling.html

In [21]:
upscale_factor = 1/3.

with rasterio.open('preprocess_dem.wgs84') as dataset:
    height, width = int(dataset.height * upscale_factor), int(dataset.width * upscale_factor)
    out_shape=(dataset.count,
               height,
               width)
    geocode_dem = dataset.read(out_shape=out_shape,
                               resampling=Resampling.bilinear)

    # scale image transform
    transform = dataset.transform * dataset.transform.scale((dataset.width / geocode_dem.shape[-1]),
                                                            (dataset.height / geocode_dem.shape[-2])
                                                           )

In [23]:
geocode_dem_profile = dem_profile_trans.copy()
geocode_dem_profile['transform'] = transform
geocode_dem_profile['width'] = width
geocode_dem_profile['height'] = height
geocode_dem_profile

{'driver': 'ISCE', 'dtype': 'int16', 'nodata': None, 'width': 6000, 'height': 4800, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0008333333333333333, 0.0, -101.0,
       0.0, -0.0008333333333333333, 22.0), 'tiled': False}

In [24]:
with rasterio.open('geocode_dem.wgs84', 'w', **geocode_dem_profile) as ds:
    ds.write(geocode_dem)

# Clip Images

Source: https://rasterio.readthedocs.io/en/latest/topics/masking-by-shapefile.html

## Preprocess

In [25]:
import rasterio
import rasterio.mask

shapes = [json.load(open('bounds.geojson'))]
shapes

[{'type': 'Polygon',
  'coordinates': [[[-96.17730760260231, 18.068982542026124],
    [-96.17730760260231, 21.84242172762871],
    [-100.13767953028737, 21.84242172762871],
    [-100.13767953028737, 18.068982542026124],
    [-96.17730760260231, 18.068982542026124]]]}]

In [27]:
with rasterio.open("preprocess_dem.wgs84") as src:
    preprocess_dem_clip, preprocess_dem_clip_transform = rasterio.mask.mask(src, shapes, crop=True)
    preprocess_profile = src.meta

In [28]:
preprocess_profile

{'driver': 'ISCE',
 'dtype': 'int16',
 'nodata': None,
 'width': 18000,
 'height': 14400,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.0002777777777777778, 0.0, -101.0,
        0.0, -0.0002777777777777778, 22.0)}

In [31]:
preprocess_profile.update({"driver": DRIVER,
                           "height": preprocess_dem_clip.shape[1],
                           "width": preprocess_dem_clip.shape[2],
                           "transform": preprocess_dem_clip_transform})

with rasterio.open("preprocess_dem.wgs84", "w", **preprocess_profile) as dest:
    dest.write(preprocess_dem_clip)

## Geocode

In [32]:
with rasterio.open("geocode_dem.wgs84") as src:
    geocode_dem_clip, geocode_dem_clip_transform = rasterio.mask.mask(src, shapes, crop=True)
    geocode_profile = src.meta

In [33]:
geocode_profile.update({"driver": DRIVER,
                        "height": geocode_dem_clip.shape[1],
                        "width": geocode_dem_clip.shape[2],
                        "transform": geocode_dem_clip_transform})

with rasterio.open("geocode_dem.wgs84", "w", **geocode_profile) as dest:
    dest.write(geocode_dem_clip)

# Build VRT

In [35]:
!gdalbuildvrt geocode_dem.wgs84.vrt geocode_dem.wgs84

0...10...20...30...40...50...60...70...80...90...100 - done.


In [36]:
!gdalbuildvrt preprocess_dem.wgs84.vrt preprocess_dem.wgs84

0...10...20...30...40...50...60...70...80...90...100 - done.


In [37]:
fix_cmd = [f"{isce_apps_path}/fixImageXml.py",
           "-i", 'preprocess_dem.wgs84', 
           "--full"
           ]
fix_cmd_line = " ".join(fix_cmd)
fix_cmd_line

'/u/leffe-data2/cmarshak/miniconda3/envs/isce2/lib/python3.8/site-packages/isce/applications/fixImageXml.py -i preprocess_dem.wgs84 --full'

In [38]:
check_call(fix_cmd_line, shell=True)

0

In [39]:
fix_cmd = [f"{isce_apps_path}/fixImageXml.py",
           "-i", 'geocode_dem.wgs84', 
           "--full"
           ]
fix_cmd_line = " ".join(fix_cmd)
fix_cmd_line

'/u/leffe-data2/cmarshak/miniconda3/envs/isce2/lib/python3.8/site-packages/isce/applications/fixImageXml.py -i geocode_dem.wgs84 --full'

In [40]:
check_call(fix_cmd_line, shell=True)

0

# Remove Old DEM Files

In [34]:
demLat_files = list(Path('.').glob('demLat_*'))
demLat_files

[PosixPath('demLat_N18_N22_Lon_W101_W096.dem'),
 PosixPath('demLat_N18_N22_Lon_W101_W096.dem.wgs84.xml'),
 PosixPath('demLat_N18_N22_Lon_W101_W096.dem.wgs84.vrt'),
 PosixPath('demLat_N18_N22_Lon_W101_W096.dem.xml'),
 PosixPath('demLat_N18_N22_Lon_W101_W096.dem.wgs84'),
 PosixPath('demLat_N18_N22_Lon_W101_W096.dem.vrt')]

In [31]:
for path in demLat_files:
    path.unlink()